# LAB SETTING

## EXPERIMENT 1: Sniffing the Network

In [23]:
# Gathering data from data/exp1
# TODO: make the text bigger for axis & more rectangualr graphs
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import statistics as st
import pprint
import json 

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots 
    "font.sans-serif": [],              # to inherit fonts from the document
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts 
    "xtick.labelsize": 8,               # a little smaller
    "ytick.labelsize": 8,
    "pgf.preamble": "\n".join([ # plots will use this preamble
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage[detect-all,locale=US]{siunitx}",
        ])
    }
mpl.use('pgf')
plt.rcParams.update(pgf_with_latex)

def get_file_name(sniffer_type, antenna, cable_length):
    return 'data/exp1/' + sniffer_type + '_sniffer_' + antenna + '_' + cable_length + 'm_cable.json'

def inner_dict_stats(dicts):
    # Step 1: Create a dictionary to hold lists of values for each key
    all_keys = [key for d in dicts for key in d.keys()]
    values_dict = {key: [] for key in all_keys}

    # Step 2: Iterate over all dictionaries and for each key, append the value to the corresponding list
    for d in dicts:
        for key, value in d.items():
            values_dict[key].append(value)

    # Step 3: Calculate the mean and standard deviation for each list
    stats_dict = {}
    for key, values in values_dict.items():
        if len(values) < 2:
            stats_dict[key] = (values[0], 0, values[0])
        else:
            stats_dict[key] = (st.mean(values), st.stdev(values), st.median(values))
    return stats_dict

def get_statistics_on_data():
    statistics_data = {}
    for sniffer_type in ['wired', 'wireless']:
        statistics_data[sniffer_type] = {}
        for cable_length in ['1', '3', '5']:
            statistics_data[sniffer_type][cable_length] = {}
            for antenna in ['blackantenna', 'purplewires']:
                statistics_data[sniffer_type][cable_length][antenna] = {}
                with open(get_file_name(sniffer_type, antenna, cable_length)) as json_file:
                    data = json.load(json_file)
                    temp_stats = {}
                    for distance in data.keys():
                        temp_stats[distance] = {}
                        for key in data[distance].keys():
                            if key == 'beacon_count_per_mac' or key == 'delimiter_count':
                                temp_stats[distance][key] = {}
                                temp_stats[distance][key] = inner_dict_stats(data[distance][key])
                            elif key == 'detected_devices':
                                pass
                            else:
                                value = data[distance][key]
                                temp_stats[distance][key] = (st.mean(value), st.stdev(value), st.median(value))

                    statistics_data[sniffer_type][cable_length][antenna] = temp_stats
    return statistics_data

data = get_statistics_on_data()

homeplug_packets_count_tuples = {}

for k1, experiments in data.items():
    homeplug_packets_count_tuples[k1] = {}
    for k2, cable_length in experiments.items():
        homeplug_packets_count_tuples[k1][k2] = {}
        for k3, distances in cable_length.items():
            homeplug_packets_count_tuples[k1][k2][k3] = {}
            for k4, metrics in distances.items():
                homeplug_packets_count_tuples[k1][k2][k3][k4] = {}
                homeplug_packets_count = metrics.get('homeplug_packets_count', None)
                if homeplug_packets_count:
                    homeplug_packets_count_tuples[k1][k2][k3][k4] = {'mean': homeplug_packets_count[0], 'std_dev': homeplug_packets_count[1], 'median': homeplug_packets_count[2]}
                    
#pprint.pprint(homeplug_packets_count_tuples)



In [24]:
import matplotlib.pyplot as plt

# Define color scheme for antennas
color_scheme = {
    'blackantenna': 'blue',
    'purplewires': 'purple', 
    'witness': 'black'    
}


# Plot for each cable length
for cable_length, cable_data in homeplug_packets_count_tuples['wireless'].items():
    plt.figure(figsize=(6, 4))

    # List to store legend handles and labels
    legend_handles = []
    legend_labels = []

    # Add "Wireless" heading
    legend_handles.append(plt.Line2D([0], [0], linestyle="none", marker="none"))
    legend_labels.append("Wireless Sniffing")
    
    # Plot for each antenna type under "Wireless"
    for antenna_type, data_points in cable_data.items():
        antenna_print_name = ""
        if antenna_type == 'blackantenna':
            antenna_print_name = r'Whip Antenna (\si{40\cm})'
        elif antenna_type == 'purplewires':
            antenna_print_name = r'Dipole Antenna (\si{10\metre})'

        distances = [float(d) for d in data_points.keys()]
        mean_values = [data['mean'] for data in data_points.values()]
        std_dev_values = [data['std_dev'] for data in data_points.values()]

        # Plot each antenna type with error bars for standard deviation, adjust line width with `linewidth`
        
        line, _, _ = plt.errorbar(distances, mean_values, yerr=std_dev_values,
                                  label=antenna_print_name,
                                  color=color_scheme[antenna_type],
                                  fmt='-', linewidth=1)  # Adjusted line width here
        legend_handles.append(line)
        legend_labels.append(antenna_print_name)

    # Add "Wired" heading
    legend_handles.append(plt.Line2D([0], [0], linestyle="none", marker="none"))
    legend_labels.append("Wired Sniffing (Control Values)")

    # Plot "Wired" data, adjust line width with `linewidth`
    # witness_data = homeplug_packets_count_tuples['wired'][cable_length][antenna_type]['0.5']   
    # witness_mean = [witness_data['mean']] * len(distances)
    # witness_std_dev = [witness_data['std_dev']] * len(distances)
    witness_mean = []
    witness_std_dev = []
    for distance in homeplug_packets_count_tuples['wired'][cable_length][antenna_type].keys():
        witness_data = homeplug_packets_count_tuples['wired'][cable_length][antenna_type][distance]   
        witness_mean.append(witness_data['mean'])
        witness_std_dev.append(witness_data['std_dev'])
    line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],
                              label='Devolo WiFi Extender',
                              color=color_scheme['witness'],
                              fmt='-', linestyle='--', linewidth=1)  # Adjusted line width here
    
    legend_handles.append(line)
    legend_labels.append(r'Devolo WiFi Extender')

    plt.xlabel(r'Distance $d$ (\si{\metre}) ', fontsize=30, labelpad=10) # 
    plt.ylabel(r'HP Packet Rate (pkts/\si{\minute}) ', fontsize=25, labelpad=10) # 
    plt.ylim(0, 2000)
    plt.tick_params(axis='both', which='major', labelsize=14)  # Adjust tick label size here
    # plt.subplots_adjust(left=0.2, right=0.8, bottom=0.2, top=0.8)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.1)

    plt.grid(True, linestyle='--')

    # Create the legend with two columns, and vertical group labels
    #leg = plt.legend(handles=legend_handles, labels=legend_labels, loc='upper left', bbox_to_anchor=(1, 1), ncol=2)

    # Customize the legend to adjust titles
    # for text, label in zip(leg.get_texts(), legend_labels):
    #     if label in ['Wireless', 'Wired']:
    #         text.set_fontsize('large')  # Keep larger font size but remove bold
    #         text.set_ha('left')  # Set horizontal alignment to left for headings

    # Save the figure
    plt.savefig(f'graphs/exp1/HP_Packet_Rate_vs_Cable_{cable_length}.pdf', bbox_inches='tight', backend="pgf")


/tmp/ipykernel_17639/944425973.py:58: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],
/tmp/ipykernel_17639/944425973.py:58: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],
/tmp/ipykernel_17639/944425973.py:58: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],


## EXPERIMENT 2: Actively Pinging

In [26]:
import os

def get_file_name(antenna, cable_length, distance, machine):
    return f'data/exp2/wireless_ping_{antenna}_{cable_length}m_cable_{distance}cm_machine{machine}.txt'

def get_last_line(antenna, cable_length, distance, machine):
    file_name = get_file_name(antenna, cable_length, distance, machine)
    if os.path.exists(file_name):
        with open(file_name) as f:
            lines = f.read().splitlines()
            last_line = lines[-2].split(',')
            return (float(last_line[1][1: last_line[1].find(' received')])/60)*100
    else:
        return 0

# Cable lengths to iterate over
cable_lengths = ['1', '3', '5']
antenna_types = ['blackantenna', 'purplewires']
distances = ['5', '10', '20', '30', '40', '70', '90', "100"]
machines = ['200', '100']

for cable_length in cable_lengths:
    # Create a new figure for each cable length
    fig, ax = plt.subplots(figsize=(6, 4))

    # Get data for each machine and antenna type
    data = {}
    for antenna in antenna_types:
        for machine in machines:
            key = f'{antenna}_machine_{machine}'
            data[key] = [get_last_line(antenna, cable_length, d, machine) for d in distances]

    # Define colors and labels
    antenna_colors = {'blackantenna_machine_100': 'dimgray', 'blackantenna_machine_200': 'gray',
                      'purplewires_machine_100': 'darkviolet', 'purplewires_machine_200': 'mediumblue'}  

    descriptive_labels = {'blackantenna_machine_100': 'Machine B',
                          'blackantenna_machine_200': 'Machine A',
                          'purplewires_machine_100': 'Machine B',
                          'purplewires_machine_200': 'Machine A'}

    # Plot data as line graphs
    for key, values in data.items():
        print(key, values)
        if any(values):
            if key == 'purplewires_machine_100' or key == 'purplewires_machine_200':
                distances.append('100')
            elif '100' in distances:
                distances.remove('100')
            label = descriptive_labels.get(key, key)
            ax.plot(distances, values, label=label, color=antenna_colors[key], linestyle='-')

    # Set titles and labels
    # ax.set_title(f'Ping Success Rate vs Distance from each Machine for a ' + cable_length + '-Meter Cable', fontsize=16)
    ax.set_xlabel(r'Distance $d$ (\si{\cm})', fontsize=30)
    ax.set_ylabel(r'Ping Success Rate (\si{\percent})', fontsize=25)
    # ax.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=2, fontsize=14)
    ax.set_ylim(0, 110)  

    # Set x-ticks
    ax.set_xticks(np.arange(len(distances)))
    ax.set_xticklabels(distances)
    ax.tick_params(axis='both', which='major', labelsize=14)  # Adjust tick label size here

    plt.grid(True, linestyle='--')  # Add grid
    plt.tight_layout()
    plt.savefig(f'graphs/exp2/Ping_Success_Rate_vs_Cable_{cable_length}.pdf', bbox_inches='tight')  # Save figure


ValueError: x and y must have same first dimension, but have shapes (9,) and (8,)

# REAL-WORLD SETTING

## EXPERIMENT 1

In [ ]:
# Gathering data from data/exp1

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics as st
import pprint
import json 

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots 
    "font.sans-serif": [],              # to inherit fonts from the document
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts 
    "xtick.labelsize": 8,               # a little smaller
    "ytick.labelsize": 8,
    "pgf.preamble": "\n".join([ # plots will use this preamble
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage[detect-all,locale=DE]{siunitx}",
        ])
    }
plt.rcParams.update(pgf_with_latex)

def get_file_name(sniffer_type, antenna):
    return 'data/world_exp1/' + sniffer_type + '_sniffer_' + antenna + '_outside.json'

def inner_dict_stats(dicts):
    # Step 1: Create a dictionary to hold lists of values for each key
    all_keys = [key for d in dicts for key in d.keys()]
    values_dict = {key: [] for key in all_keys}

    # Step 2: Iterate over all dictionaries and for each key, append the value to the corresponding list
    for d in dicts:
        for key, value in d.items():
            values_dict[key].append(value)

    # Step 3: Calculate the mean and standard deviation for each list
    stats_dict = {}
    for key, values in values_dict.items():
        if len(values) < 2:
            stats_dict[key] = (values[0], 0, values[0])
        else:
            stats_dict[key] = (st.mean(values), st.stdev(values), st.median(values))
    return stats_dict

def get_statistics_on_data():
    statistics_data = {}
    for sniffer_type in ['wired', 'wireless']:
        statistics_data[sniffer_type] = {}
        for antenna in ['blackantenna', 'purplewires']:
            statistics_data[sniffer_type][antenna] = {}
            with open(get_file_name(sniffer_type, antenna)) as json_file:
                data = json.load(json_file)
                temp_stats = {}
                for distance in data.keys():
                    temp_stats[distance] = {}
                    for key in data[distance].keys():
                        if key == 'beacon_count_per_mac' or key == 'delimiter_count':
                            temp_stats[distance][key] = {}
                            temp_stats[distance][key] = inner_dict_stats(data[distance][key])
                        elif key == 'detected_devices':
                            pass
                        else:
                            value = data[distance][key]
                            temp_stats[distance][key] = (st.mean(value), st.stdev(value), st.median(value))

                statistics_data[sniffer_type][antenna] = temp_stats
    return statistics_data

data = get_statistics_on_data()

homeplug_packets_count_tuples = {}

for k1, experiments in data.items():
    homeplug_packets_count_tuples[k1] = {}
    for k3, distances in experiments.items():
        homeplug_packets_count_tuples[k1][k3] = {}
        for k4, metrics in distances.items():
            homeplug_packets_count_tuples[k1][k3][k4] = {}
            homeplug_packets_count = metrics.get('homeplug_packets_count', None)
            if homeplug_packets_count:
                homeplug_packets_count_tuples[k1][k3][k4] = {'mean': homeplug_packets_count[0], 'std_dev': homeplug_packets_count[1], 'median': homeplug_packets_count[2]}
                    
# pprint.pprint(homeplug_packets_count_tuples)



In [ ]:
import matplotlib.pyplot as plt

# Define color scheme for antennas
color_scheme = {
    'blackantenna': 'blue',
    'purplewires': 'purple', 
    'witness': 'black'    
}


plt.figure(figsize=(8, 5))

# List to store legend handles and labels
legend_handles = []
legend_labels = []

# Add "Wireless" heading
legend_handles.append(plt.Line2D([0], [0], linestyle="none", marker="none"))
legend_labels.append("Wireless Sniffing")

# Plot for each antenna type under "Wireless"
for antenna_type, data_points in homeplug_packets_count_tuples['wireless'].items():
    antenna_print_name = ""
    if antenna_type == 'blackantenna':
        antenna_print_name = r'Whip Antenna (\si{40\cm})'
    elif antenna_type == 'purplewires':
        antenna_print_name = r'Dipole Antenna (\si{10\metre})'

    distances = [float(d) for d in data_points.keys()]
    mean_values = [data['mean'] for data in data_points.values()]
    std_dev_values = [data['std_dev'] for data in data_points.values()]

    # Plot each antenna type with error bars for standard deviation, adjust line width with `linewidth`
    line, _, _ = plt.errorbar(distances, mean_values, yerr=std_dev_values,
                                label=antenna_print_name,
                                color=color_scheme[antenna_type],
                                fmt='-', linewidth=1)  # Adjusted line width here
    legend_handles.append(line)
    legend_labels.append(antenna_print_name)

# Add "Wired" heading
legend_handles.append(plt.Line2D([0], [0], linestyle="none", marker="none"))
legend_labels.append("Wired Sniffing (Control Values)")

witness_mean = []
witness_std_dev = []
for distance in homeplug_packets_count_tuples['wired'][antenna_type].keys():
    witness_data = homeplug_packets_count_tuples['wired'][antenna_type][distance]   
    witness_mean.append(witness_data['mean'])
    witness_std_dev.append(witness_data['std_dev'])
line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],
                            label='Devolo WiFi Extender',
                            color=color_scheme['witness'],
                            fmt='-', linestyle='--', linewidth=1)  # Adjusted line width here
legend_handles.append(line)
legend_labels.append(r'Devolo WiFi Extender')

plt.xlabel(r'Distance $d$ (\si{\metre})', fontsize=30, labelpad=10)
plt.ylabel(r'HP Packet Rate (pkts/\si{\minute})', fontsize=25, labelpad=10)
plt.ylim(0, 2000)
plt.tick_params(axis='both', which='major', labelsize=14)  # Adjust tick label size here

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)

plt.grid(True, linestyle='--')
# Create the legend with two columns, and vertical group labels
# leg = plt.legend(handles=legend_handles, labels=legend_labels, loc='upper left', bbox_to_anchor=(1, 1), ncol=2)

# # Customize the legend to adjust titles
# for text, label in zip(leg.get_texts(), legend_labels):
#     if label in ['Wireless', 'Wired']:
#         text.set_fontsize('large')  # Keep larger font size but remove bold
#         text.set_ha('left')  # Set horizontal alignment to left for headings

# Save the figure
plt.savefig(f'graphs/world_exp1/HP_Packet_Rate_real_world.pdf', bbox_inches='tight')


/tmp/ipykernel_17639/847606824.py:51: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-" (-> linestyle='-'). The keyword argument will take precedence.
  line, _, _ = plt.errorbar(distances, witness_mean[0:len(distances)], yerr=witness_std_dev[0:len(distances)],


## EXPERIMENT 2

In [ ]:
import os

def get_file_name(distance, machine, environment):
    return f'data/world_exp2/wireless_ping_purplewires_{distance}cm_{environment}_machine{machine}.txt'

def get_last_line(distance, machine, environment):
    file_name = get_file_name(distance, machine, environment)
    if os.path.exists(file_name):
        with open(file_name) as f:
            lines = f.read().splitlines()
            last_line = lines[-2].split(',')
            return (float(last_line[1][1: last_line[1].find(' received')])/60)*100
    else:
        return 0

# Cable lengths to iterate over
environments = ['inside', 'outside']
antenna_types = ['purplewires']
distances = ['0', '20', '50', '100', '150', '200']
machines = ['200', '100']

for environment in environments:
    fig, ax = plt.subplots(figsize=(6, 4))

    # Get data for each machine and antenna type
    data = {}
    for machine in machines:
        key = f'purplewires_machine_{machine}'
        data[key] = [get_last_line(d, machine, environment) for d in distances]

    # Define colors and labels
    antenna_colors = {'purplewires_machine_100': 'darkviolet', 'purplewires_machine_200': 'mediumblue'}  

    descriptive_labels = {'purplewires_machine_100': 'Machine B',
                            'purplewires_machine_200': 'Machine A'}

    # Plot data as line graphs
    for key, values in data.items():
        if any(values):
            label = descriptive_labels.get(key, key)
            ax.plot(distances, values, label=label, color=antenna_colors[key], linestyle='-')

    # Set titles and labels
    ax.set_xlabel(r'Distance $d$ (\si{\cm})', fontsize=30)
    ax.set_ylabel(r'Ping Success Rate (\si{\percent})', fontsize=25)
    # ax.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=2, fontsize=14)
    ax.set_ylim(0, 110)  

    # Set x-ticks
    ax.set_xticks(np.arange(len(distances)))
    ax.set_xticklabels(distances)

    ax.tick_params(axis='both', which='major', labelsize=14)  # Adjust tick label size here
    plt.grid(True, linestyle='--') 
    plt.tight_layout()
    plt.savefig(f'graphs/world_exp2/Ping_Success_Rate_real_world_{environment}.pdf', bbox_inches='tight')  # Save figure


/tmp/ipykernel_17639/3433624024.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(6, 4))
